<a href="https://colab.research.google.com/github/omarShiraz/chatbotLawyer/blob/main/Domain_Specific_Dataset_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --verbose git+https://github.com/omarShiraz/Questgen.ai.git

In [ ]:
!pip install fitz
!pip install PyMuPDF
!pip install transformers
!pip install --upgrade numpy
!pip install spaCy==2.3.3
!pip install --quiet git+https://github.com/boudinfl/pke.git
!python -m nltk.downloader universal_tagset
!python -m spacy download en_core_web_sm

## **Restart runtime before continuing**

In [ ]:
!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz
!tar -xvf  s2v_reddit_2015_md.tar.gz
!ls s2v_old

In [ ]:
import nltk
from pprint import pprint
from Questgen import main
qg = main.QGen()

In [3]:
import fitz  # PyMuPDF
import transformers
from transformers import AutoTokenizer,BertTokenizerFast, BertForQuestionAnswering
import torch
import pandas as pd
# Define the bert tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Load the fine-tuned modeol
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

def generate_answer(question, context):

  inputs = tokenizer.encode_plus(question, context, return_tensors='pt')

  outputs = model(**inputs)
  answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
  answer_end = torch.argmax(outputs[1]) + 1

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

  return answer

  # Encode the input text and question, and get the scores for each word in the text


# Find the words in the text that corresponds to the highest start and end scores
# with a torch.no_grad():
#     outputs = model(**inputs)


# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_file):
    doc = fitz.open(pdf_file)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Function to chunk text into pieces of a specified size
def chunk_text(text, chunk_size=4000):
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks

# Function to preprocess text (you can customize this)
def preprocess_text(text):
    # Remove extra whitespace and special characters
    text = ' '.join(text.split())
    return text

# Load your PDF file
pdf_file_path = '/content/SCHCCALA99of2008.pdf'

# Extract text from the PDF
pdf_text = extract_text_from_pdf(pdf_file_path)

# Preprocess the text
cleaned_text = preprocess_text(pdf_text)

# Chunk the text into 2000-token pieces
text_chunks = chunk_text(cleaned_text, chunk_size=512)

payload = {"input_text": ""}
question_context_pairs = []  # Store question-context pairs
i = 1
# Initialize a list to store the generated answers with context
answers_with_context = []

# Loop through chunks and make predictions
for chunk in text_chunks:
    payload["input_text"] = chunk
    outputs = qg.predict_shortq(payload)

    # Check the structure of the outputs dictionary
    if 'questions' in outputs:
        question_list = outputs['questions']
    elif 'your_custom_key' in outputs:
        question_list = outputs['your_custom_key']
    else:
        # Handle the case when the structure is different
        print("Unexpected structure in the 'outputs' dictionary. Check the structure and update the code.")
        continue

    # Iterate through the extracted questions and contexts
    for item in question_list:
        question = item.get('Question', '')  # Use get to avoid KeyError
        context = item.get('context', '')  # Use get to avoid KeyError

        # Check if question and context are non-empty before processing
        if question and context:
            question_context_pairs.append((question, context))
            answer = generate_answer(question, context)

            # Include context in the instruction field
            instruction_text = f"You are a problem-solving assistant. Before answering, explain your reasoning step-by-step. Context: {context}\nFinal answer: {answer}"

            answers_with_context.append({
                "input": f"Question: {question}\nContext: {context}",
                "output": answer,
                "instruction": instruction_text
            })

# Convert the list of answers with context to a DataFrame
answers_df = pd.DataFrame(answers_with_context)

# Save the DataFrame to your desired format (e.g., CSV)
answers_df.to_csv('output_dataset.csv', index=False)

# Print the generated answers with context
print(answers_df)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Running model for generation
{'questions': [{'Question': 'What is the socialist republic of SRI LANKA?', 'Answer': 'socialist republic', 'id': 1, 'context': '1 IN THE SUPREME COURT OF THE DEMOCRATIC SOCIALIST REPUBLIC OF SRI LANKA ------------------------------------------------------------------ S.C. (HC) CA LA No.'}]}
ZERO
Unexpected structure in the 'outputs' dictionary. Check the structure and update the code.
Running model for generation
{'questions': [{'Question': 'What court did the plaintiff-appellant appeal from the judgment of the Western Province (Civil Appeals) holden at Gampaha dated 15.07.2008?', 'Answer': 'high court', 'id': 1, 'context': 'This is an application for leave to appeal from the judgment of the High Court of the Western Province (Civil Appeals) holden at Gampaha dated 15.07.2008. By that judgment the learned Judges of the High Court had dismissed the appeal of the plaintiff-appellant, now deceased.'}, {'Question': 'What is the name of the High Court of Wester